In [1]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index
%cd ..

# Import Libraries

In [2]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
import time
sys.path.append('../input/tensorflow-great-barrier-reef')
import torch
from PIL import Image, ImageDraw
import ast
import albumentations as albu

In [3]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'

# Increase Annotations

**Hypothesis:** we can enlarge our training data set by adding earlier bounding boxes prior to the first detected box for each COTS

An increase in annotations means increase in the target variable which hypotheticaly can increase accuracy.

**Approach:**

1. Add detections to earlier frames that have a detection in a subsequent frame as follows:
* identify frames that have less detections than the next one
* exclude any candidate boxes if the box has an overlap with another box in the previous frame
* exclude the image "margins" to account for that some boxes may not have been visible in the frame
* shift the bounding box by the average translation of any other matched boxes
2. Compare some examples frames right before and right after a detection
3. Save the results into a new training set

In [4]:
#Parameters
IMAGE_DIM = (1280,720)
#we will not be adding boxes in 5% of the image..
#..width or height
EXCLUDE_MARGIN = 0.05

## Data Extraction Helper functions

In [5]:
def get_bbox(annots):
    """Get a list of bounding boxes by excluding key part 
    of their dict and extracting only actual values."""
    #extract bounding box coordinates and dimentions
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def read_data():
    """Get all the data required/provided."""
    
    #read the train csv
    df_train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
    df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
    #Safely evaluate annotation encoded string containing a Python expression.
    df_train['annotations'] = df_train['annotations'].apply(lambda x: ast.literal_eval(x))
    df_train['bboxes'] = df_train['annotations'].apply(lambda x: get_bbox(x))
    #get number of bounding boxes
    df_train['Number_bbox'] = df_train['annotations'].apply(lambda x:len(x))
    return df_train

df_train = read_data()

## Add New Bounding Boxes to Previous Frames
We shift current annotation to previous frame and use frames that have less
annonations than the next as they are the ones most likely to have starfish
show before its actually annotated.

In [6]:
#shift next annotations to previous frame and rename columns to reflect that.
df_shift = df_train.shift(-1).rename(columns={'annotations':'annotations_n1',
                                             'Number_bbox':'Number_bbox_n1',
                                             'img_path':'img_path_n1'})
df_lagged = pd.concat([df_train, df_shift], axis=1)

#identify frames that have less annotations than the next one
#these are the candidates for adding earlier annotations
df_first_frames = df_lagged[df_lagged.Number_bbox < df_lagged.Number_bbox_n1]

In [7]:
df_first_frames.head()

In [8]:
def intersects(rectangle_a, rectangle_b):
    '''Checks for intersection of two rectangles specified as [(x1,y1),(x2,y2)]'''
    if(rectangle_a[1][0]<rectangle_b[0][0] or rectangle_a[1][1]<rectangle_b[0][1]):
        return False
    elif(rectangle_a[0][0]>rectangle_b[1][0] or rectangle_a[0][1]>rectangle_b[1][1]):
        return False
    else:
        return True
        
def new_bboxes(prev_bboxes, next_bboxes):
    '''Returns the bounding boxes that are deemed new in the next frame by checking 
    the centers of the bounding box in the next frame are not contained in
    one of the previous frame bounding boxes.'''
    new_bbs =[]
    delta_xs = [0]
    delta_ys = [0]
    delta_ws = [0]
    delta_hs = [0]
    for bb in next_bboxes:
        found = False
        for prev_bb in prev_bboxes:
            if intersects([(bb['x'],bb['y']),(bb['x'] + bb['width'],bb['y'] + bb['height'])],
                         [(prev_bb['x'], prev_bb['y']), (prev_bb['x'] + prev_bb['width'], 
                                                         prev_bb['y'] + prev_bb['height'])]
                         ):
                delta_xs.append(bb['x']-prev_bb['x'])
                delta_ys.append(bb['y']-prev_bb['y'])
                delta_ws.append(bb['width']-prev_bb['width'])
                delta_hs.append(bb['height']-prev_bb['height'])
                found = True
                break
        if found == False:
            #exclude margins
            if (bb['x'] > IMAGE_DIM[0]*EXCLUDE_MARGIN) & \
            (bb['x'] < (IMAGE_DIM[0]-IMAGE_DIM[0]*EXCLUDE_MARGIN)) & \
            (bb['y'] > IMAGE_DIM[1]*EXCLUDE_MARGIN) & \
            (bb['y'] < (IMAGE_DIM[1]-IMAGE_DIM[1]*EXCLUDE_MARGIN)):
                new_bb = {'x': bb['x'], 'y': bb['y'], 'width':bb['width'], 'height':bb['height']}
                new_bbs.append(new_bb)
                
    #adjust bounding boxes for avergage drift
    for b in new_bbs:        
        delta_x_avg = sum(delta_xs)/len(delta_xs)
        delta_y_avg = sum(delta_ys)/len(delta_ys)
        delta_w_avg = sum(delta_ws)/len(delta_ws)
        delta_h_avg = sum(delta_hs)/len(delta_hs)
        b['x'] = b['x'] + delta_x_avg
        b['y'] = b['y'] + delta_y_avg
               
    return new_bbs

In [9]:
# create new column to store only bounding boxes that are new on the next frame.
df_first_frames['new_annotations'] = df_first_frames.apply(lambda x: 
                                                            new_bboxes(x['annotations'],
                                                                      x['annotations_n1']),
                                                          axis=1)

In [10]:
#Visualize new bounding boxes
def viz_new_boxes(prev_path, next_path, prev_annots, next_annots, new_annots):  
    """Draws actual/original bboxes(red) and potential new annotations(yellow)"""
    #previuos frame
    print(prev_path)
    img = Image.open(prev_path)
    
    #draw red box for the previous annotations
    for box in prev_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)

    #draw yellow box for the new annotations
    for box in new_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="yellow", width=3)

    display(img)    
    
    #next frame
    print(next_path)
    img = Image.open(next_path)
    #On the next frame draw red boxes this include boxes previously denoted as yellow
    for box in next_annots:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
        
    display(img)

In [11]:
#Get a sample of 10 consecutive frames and apply visualization
for index, row in df_first_frames.sample(10, random_state=12).iterrows():
    viz_new_boxes(row.img_path,
                  row.img_path_n1,
                  row.annotations,
                  row.annotations_n1,
                  row.new_annotations)

All the new box previously maked as yellow show on the next frame as red which means they are not new on that specific frame. This means we have succefully shifted/generated new bounding boxes.

In [12]:
#Get new annotation column to merge with original dataset
df_first_frames_strip = df_first_frames[['new_annotations']]
df_first_frames_strip.head(2)

In [14]:
#Join new annotations with original dataset
df_train_new = df_train.join(df_first_frames_strip)
df_train_new.head(2)

In [15]:
#Replace NaN with empty list [] as in the original dataset
df_train_new['new_annotations'].loc[df_train_new['new_annotations'].isnull()] = df_train_new['new_annotations'].loc[df_train_new['new_annotations'].isnull()].apply(lambda x: []) 

In [16]:
#Merge original annotations with new annotations.
df_train_new['merge_annotations'] = df_train_new.apply(lambda x: (x['annotations'] + x['new_annotations']),axis=1)

In [17]:
df_train_new.head(2)

In [18]:
#Get total number of original annotations
(df_train_new['annotations'].apply(lambda x:len(x))).sum()

In [19]:
#Get total number of new annotations
(df_train_new['new_annotations'].apply(lambda x:len(x))).sum()

In [20]:
#Get total number of annotations combined
(df_train_new['merge_annotations'].apply(lambda x:len(x))).sum()

In [22]:
#Number of new annotations per row
df_train_new['New_number_bbox'] = df_train_new['merge_annotations'].apply(lambda x:len(x))

In [23]:
# Short report of the entire process
prev_box_count = df_train_new['Number_bbox'].sum()
curr_box_count = df_train_new['New_number_bbox'].sum()
prev_frames_with_box_count = df_train_new[df_train_new.Number_bbox >0]['video_id'].count()
curr_frames_with_box_count = df_train_new[df_train_new.New_number_bbox >0]['video_id'].count()
print("Previous number of bounding boxes: ", prev_box_count)
print("New number of boxes: ", curr_box_count)
print("Number of boxes increase: ", curr_box_count-prev_box_count)
print("Previous number of frames with boxes: ", prev_frames_with_box_count)
print("New number of frames with boxes: ", curr_frames_with_box_count)
print("Number of frames with boxes increase: ", curr_frames_with_box_count-prev_frames_with_box_count)

There is an increase in the number of annotations. However there is an opportunity to get more
annotations if we shift more frames backwards. [MACITATA](https://www.kaggle.com/bartmaciszewski) (on kaggle) also suggests "Using 'optical flow' techniques to determine where the boxes may have come from better, fit tighter boxes, and project boxes into margins if possible". For now we will conclude.

In [24]:
#Replace original annotations column with merge_annotations
df_train['annotations'] = df_train_new['merge_annotations']

In [ ]:
# Get a copy of train dataset and prepare for ML
df = df_train.copy()
df['image_path'] = f'/kaggle/input/tensorflow-great-barrier-reef/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
#df['annotations'] = df['annotations'].progress_apply(eval)
display(df.head(2))

In [ ]:
FDA_reference = df[df['annotations']!='[]']

In [ ]:
FDA_trans = albu.FDA(FDA_reference['image_path'].values)

## Number of BBoxes

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts()/len(df)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

# 🔨 Helper

In [ ]:
def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [ ]:
##############################################################
#                      Tracking helpers                      #
##############################################################

import numpy as np
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)



In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
# def load_model(ckpt_path, conf=0.28, iou=0.40):
def load_model(ckpt_path, conf=0.28, iou=0.40):
    model = torch.hub.load('/kaggle/input/yolov5-lib-ds',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 20  # maximum number of detections per image
    return model

# 🔭 Inference

## Helper

In [ ]:
def predict(model, img, size=9000, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [ ]:
def tracking_function(tracker, frame_id, bboxes, scores):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]

        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        
    return predictions

## Run Inference on **Train**

In [ ]:
CKPT_PATH = '../input/yolov5s6/f2_sub2.pt'
IMG_SIZE  = 6400
CONF      = 0.30
IOU       = 0.50
AUGMENT   = False
FDA_aug = False

In [ ]:
def CLAHE(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized = clahe.apply(gray)
    return equalized
def Gamma_enhancement(image):
    gamma = 1/0.6
    R = 255.0
    return (R * np.power(image.astype(np.uint32)/R, gamma)).astype(np.uint8)


In [ ]:
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
image_paths = df[df.num_bbox>1].sample(100).image_path.tolist()
frame_id = 0
for idx, path in enumerate(image_paths):
    img = cv2.imread(path)[...,::-1]
    if FDA_aug:
        img = FDA_trans(image=img)['image']
    bboxes, confis = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    predict_box = tracking_function(tracker, frame_id, bboxes, confis)

    if len(predict_box)>0:
        box = [list(map(int,box.split(' ')[1:])) for box in predict_box]
    else:
        box = []
    display(show_img(img, box, bbox_format='coco'))
    if idx>5:
        break
    frame_id += 1

## Init `Env`

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
cd ../working

## Run Inference on **Test**

In [ ]:
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)

model = load_model(CKPT_PATH, conf=CONF, iou=IOU)

frame_id =0
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    if FDA_aug:
        img = FDA_trans(image=img)['image']
    bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=AUGMENT)

    predictions = tracking_function(tracker, frame_id, bboxes, confs)
    
    prediction_str = ' '.join(predictions)
    pred_df['annotations'] = prediction_str
    env.predict(pred_df)
    if frame_id < 3:
        if len(predict_box)>0:
            box = [list(map(int,box.split(' ')[1:])) for box in predictions]
        else:
            box = []
        display(show_img(img, box, bbox_format='coco'))
#     print('Prediction:', pred_df)
    frame_id += 1


# 👀 Check Submission

In [ ]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()